In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import numpy as np
import queue
%matplotlib inline

In [7]:
tp = pd.read_csv('capture20110818.pcap.netflow.labeled.csv', chunksize=1000)
df = pd.concat(tp, ignore_index=True)

# preprocessing
ips = df['SrcIPAddr:Port'].str.split(':')
df['SrcIPAddr'] = ips.str[0]
df['SrcIPPort'] = ips.str[-1]
ips = df['DstIPAddr:Port'].str.split(':')
df['DstIPAddr'] = ips.str[0]
df['DstIPPort'] = ips.str[-1]

# # df.dtypes
# ips = df['SrcIPAddr:Port'].str.split(':')
# print(ips)

# # print(pd.sort(ips[0:-1]))
# df['SrcIPAddr'] = (ips.str[0:-1])
# df['SrcIPPort'] = ips.str[-1]
# ips = df['DstIPAddr:Port'].str.split(':')
# df['DstIPAddr'] = ':'.join(ips[0:-1])
# df['DstIPPort'] = ips.str[-1]
df.head()

,Date,Flow_start,Durat,Prot,SrcIPAddr:Port,tmp,DstIPAddr:Port,Flags,Tos,Packets,Bytes,Flows,Label,Labels,SrcIPAddr,SrcIPPort,DstIPAddr,DstIPPort
0,2011-08-18,10:19:13.328,0.002,TCP,147.32.86.166:33426,->,212.24.150.110:25443,FRPA_,0,4,321,1,Background,NaN,147.32.86.166,33426,212.24.150.110,25443
1,2011-08-18,10:19:13.328,4.995,UDP,82.39.2.249:41915,->,147.32.84.59:43087,INT,0,617,40095,1,Background,NaN,82.39.2.249,41915,147.32.84.59,43087
2,2011-08-18,10:19:13.329,4.996,UDP,147.32.84.59:43087,->,82.39.2.249:41915,INT,0,1290,1909200,1,Background,NaN,147.32.84.59,43087,82.39.2.249,41915
3,2011-08-18,10:19:13.330,0.000,TCP,147.32.86.166:42020,->,147.32.192.34:993,A_,0,1,66,1,Background,NaN,147.32.86.166,42020,147.32.192.34,993
4,2011-08-18,10:19:13.330,0.000,TCP,212.24.150.110:25443,->,147.32.86.166:33426,FPA_,0,2,169,1,Background,NaN,212.24.150.110,25443,147.32.86.166,33426


In [8]:
#calculate frequency
df.groupby(['SrcIPAddr'])['SrcIPAddr'].agg({'Frequency':'count'}).sort_values(['Frequency'], ascending=False)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,Frequency
SrcIPAddr,
147.32.84.229,179799
147.32.80.9,109284
147.32.84.59,101036
147.32.84.138,49379
147.32.84.118,12724
147.32.85.25,10381
147.32.80.13,9064
147.32.84.95,8592
147.32.86.20,8236


In [24]:
# rows = np.random.choice(df.index.values, 300)
# sampled_df = df.ix[rows]
# sampled_df.groupby(['SrcIPAddr'])['SrcIPAddr'].agg({'Frequency':'count'}).sort_values(['Frequency'], ascending=False)

C:\Users\Anna\AppData\Local\Programs\Python\Python36-32\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


,Frequency
SrcIPAddr,
147.32.84.229,43
147.32.80.9,37
147.32.84.59,31
147.32.84.138,11
147.32.84.118,7
147.32.85.26,6
147.32.85.34,4
147.32.87.38,3
147.32.84.164,3


In [9]:
#assign random value from a uniform distribution [0,1)
df['rnd'] = np.random.rand(len(df))
#take 300 most lowest ones
df_rnd = df.sort_values(['rnd']).head(300)
#calculate frequency
df_rnd.groupby(['SrcIPAddr'])['SrcIPAddr'].agg({'Frequency':'count'}).sort_values(['Frequency'], ascending=False)

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,Frequency
SrcIPAddr,
147.32.84.229,47
147.32.80.9,32
147.32.84.59,24
147.32.84.138,10
147.32.84.95,6
147.32.84.118,5
147.32.86.20,5
147.32.80.13,4
147.32.85.25,3


In [30]:
reservoir = queue.PriorityQueue(maxsize=300)
for index, row in df.iterrows():
    r = np.random.rand(1)
    if(reservoir.full()):
#         print("reservoir is full")
        ips = reservoir.get()
        if (ips[0]<r):
            reservoir.put((r,row['SrcIPAddr']))
#             print("added")
        else:
            reservoir.put((ips[0],ips[1]))
#             print("not added")
    else:
        reservoir.put((r,row['SrcIPAddr']))
#         print("added")

ips = {}
while not reservoir.empty():
    ip = reservoir.get()[1]
    if(ip not in ips):
        ips[ip] = 0
    ips[ip] += 1
    

In [45]:
import operator
sorted_ips = sorted(ips.items(), key=operator.itemgetter(1),reverse=True)
print(sorted_ips)

[('147.32.84.229', 55), ('147.32.84.59', 34), ('147.32.80.9', 22), ('147.32.84.138', 16), ('147.32.86.187', 5), ('147.32.84.111', 4), ('147.32.86.20', 4), ('147.32.85.26', 4), ('147.32.80.13', 3), ('147.32.84.68', 3), ('147.32.84.95', 3), ('147.32.86.50', 2), ('74.125.232.215', 2), ('147.32.86.172', 2), ('147.32.84.164', 2), ('147.32.84.118', 2), ('147.32.85.34', 2), ('147.32.85.25', 2), ('178.123.158.98', 2), ('147.32.86.194', 2), ('147.32.85.120', 2), ('141.146.44.21', 2), ('147.32.87.238', 2), ('147.32.85.60', 2), ('24.231.148.62', 1), ('147.32.84.166', 1), ('195.212.29.164', 1), ('147.32.84.143', 1), ('213.87.131.201', 1), ('178.219.32.24', 1), ('205.188.10.203', 1), ('94.252.214.83', 1), ('41.100.156.149', 1), ('81.20.48.51', 1), ('147.32.87.78', 1), ('78.45.225.28', 1), ('66.25.66.164', 1), ('147.32.84.8', 1), ('147.32.86.168', 1), ('147.32.84.194', 1), ('188.95.61.42', 1), ('194.108.42.229', 1), ('147.122.45.75', 1), ('147.32.86.223', 1), ('147.32.84.94', 1), ('147.32.87.220', 1